### Text Selection Data Merge

In [22]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [23]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [24]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [25]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Data/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [26]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [27]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [28]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [29]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [30]:
#def multi_wordgroup_in_wordgroup(word_list, df_target, target_column):
#    source_wordgroup_list = word_list
#    df_target2 = df_target
#    target_column2 = target_column
#
#    searchlist = []
#    resultlist = []
#    manager = multiprocessing.Manager()
#    resultlist = manager.list()
#    searchlist = manager.list()
#
#    def word_in_wordgroup(wordgroup):        
#        try:
#            word_in_word_cluster = df_target2[df_target2[f"{target_column2}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
#            for result in word_in_word_cluster[f"{target_column2}"]:
#                searchlist.append(wordgroup)
#                resultlist.append(result)
#        except:
#            pass
#
#    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
#        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste
#
#    search_list = list(searchlist)
#    result_list = list(resultlist)
#    df_result = pd.DataFrame([search_list,result_list]).transpose()
#    df_result.columns = ["word",f"{target_column}"]
#    df_result
#
#    return df_result

#### Text Data Read

In [31]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [32]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
213666,olmadan gerçek,52
213667,epey kolay,52
213668,hafızan iyi,52
213669,senin yaşındaydım,52


In [33]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
307731,arkadaş mı olmak,38
307732,olan birkaç şey,38
307733,da size anlatayım,38
307734,için olay yerine,38


In [34]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
388274,söz veriyorum kurul iş,16
388275,çıkmak için can atıyor,16
388276,ve içinden alevlerin çıktığı,16
388277,ne zaman isterse ama,16


In [35]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
122369,ile matrix arasında seçim yapmam,19
122370,hey o iyi bir adam,19
122371,beklemek için nasıl bir yer,19
122372,yani bak normalde beni bilirsin,19


In [36]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
300159,bu çocuğun bir süper babaya ihtiyacı,9
300160,sadece yaranın üzerine baskı uygulamaya çalıştım,9
300161,tamam bir iki hafta içinde uğra,9
300162,bana şükran gününde ananas toplama sözü,9


In [37]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9
141655,herifi durdurmak için sadece bir saat kırk,9
141656,görücü usulü evlenmek daha çok işe yarıyor,9
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9


In [38]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
109469,ve bunu her yaptığında takım arkadaşlarından b...,7
109470,gizli gizli dolaşan genç bir haham imajı canla...,7
109471,için tek yol kahrolası paramı bana geri vermen,7
109472,teşkilatın görev kapsamında olsa bile görev se...,7


In [39]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

,ninegram,frequency
0,dokuz sekiz yedi altı beş dört üç iki bir,463
1,bir iki üç dört beş altı yedi sekiz dokuz,405
2,iki üç dört beş altı yedi sekiz dokuz on,287
3,on dokuz sekiz yedi altı beş dört üç iki,277
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134
...,...,...
40393,artık sana kan nakli yapacak doktor hoffman da...,7
40394,senin ülken için potansiyel terörist teröristi...,7
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7
40396,herkes benim iyi adam olduğuma inanmakta bu ka...,7


In [40]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

,tengram,frequency
0,bir iki üç dört beş altı yedi sekiz dokuz on,287
1,on dokuz sekiz yedi altı beş dört üç iki bir,251
2,g i t a r i s y e n,128
3,t o x i c w o r l d,93
4,umutsuzca birinin arkasından koşuyorum daha gü...,76
...,...,...
30518,kendi ülken başkasının ülkesi ya da kendi kişi...,5
30519,son altı ay içinde aynı tarzda bir düzine soyg...,5
30520,benim hikayem birkaç yıl önce yayınlandı kendi...,5
30521,aslında ben bunun pek de kötü bir fikir olmadı...,5


#### Text Data Word-Word Group In Word Group

In [41]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

In [17]:
#multi_wordgroup_in_wordgroup(word_list, df_twogram_select, "twogram")

##### Word

In [18]:
# 1 Way Slow
#df_word_in_two = word_in_wordgroup(word_list, df_twogram_select, "twogram", simple=True)
#df_word_in_two

In [19]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_two

,word,twogram
0,kazanıyorum,para kazanıyorum
1,kazanıyorum,böyle kazanıyorum
2,kazanıyorum,hayatımı kazanıyorum
3,kazanıyorum,dolar kazanıyorum
4,kazanıyorum,ben kazanıyorum
...,...,...
426664,ayrıntısına,ayrıntısına kadar
426665,ayrıntısına,ince ayrıntısına
426666,kardeşisin,onun kardeşisin
426667,kardeşisin,kız kardeşisin


In [20]:
df_word_in_two["word"].nunique()

29823

In [21]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

,word,twogram
0,a,"[l a, a evet, a j, a b, a aziz, and a, a ve, t..."
1,aa,"[aa evet, şey aa, aa bu, aa şey, aa öyle, aa h..."
2,aaa,[aaa şey]
3,aah,"[aah o, aah en]"
4,aang,"[aang adında, aang adlı]"
...,...,...
29818,şık,"[çok şık, şık bir, şık görünüyorsun, kadar şık..."
29819,şımarık,"[şımarık bir, şımarık ve, seni şımarık]"
29820,şınav,[şınav çek]
29821,şırınga,[bir şırınga]


In [22]:
#df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

In [42]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_three = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_three.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_three

,word,threegram
0,geçtik,o yollardan geçtik
1,geçtik,az önce geçtik
2,geçtik,duvarı yıkıp geçtik
3,geçtik,önce de geçtik
4,geçtik,bu yollardan geçtik
...,...,...
919657,ayrıntısına,ince ayrıntısına kadar
919658,ayrıntısına,ufak ayrıntısına kadar
919659,ayrıntısına,en ufak ayrıntısına
919660,kardeşisin,kardeşisin değil mi


In [43]:
df_word_in_three_list = df_word_in_three.groupby(["word"])["threegram"].apply(list).reset_index(name="threegram")
df_word_in_three_list

,word,threegram
0,a,"[a ve b, a b c, two and a, a r i, t a r, i t a..."
1,aah,"[aah o günkü, aah en uzun, aah o gün]"
2,aang,"[aang adında bir, aang adlı bir]"
3,ab,[kan grubu ab]
4,abartma,"[kadar da abartma, bu kadar abartma, o kadar a..."
...,...,...
24427,şüphesi,[kimsenin şüphesi yok]
24428,şüphesiz,"[hiç şüphesiz bu, hiç şüphesiz ki, şüphesiz ki..."
24429,şüpheye,[şüpheye yer bırakmayacak]
24430,şık,"[çok şık bir, şık bir yer, ne kadar şık, şık b..."


In [44]:
df_word_in_three["word"].nunique()

24432

In [46]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_four

,word,fourgram
0,bildirin,gemideki herkesin öldürüldüğünü bildirin
1,bildirin,ve nereye gittiğinizi bildirin
2,bildirin,tanıtın ve pozisyonunuzu bildirin
3,bildirin,ve pozisyonunuzu bildirin tamam
4,bildirin,durumunuzu bildirin ve kendinizi
...,...,...
1544435,ayrıntısına,en ince ayrıntısına kadar
1544436,ayrıntısına,en ufak ayrıntısına kadar
1544437,ayrıntısına,şeyi en ince ayrıntısına
1544438,kardeşisin,de onun kız kardeşisin


In [47]:
df_word_in_four_list = df_word_in_four.groupby(["word"])["fourgram"].apply(list).reset_index(name="fourgram")
df_word_in_four_list

,word,fourgram
0,a,"[a b c d, i t a r, g i t a, a r i s, t a r i, ..."
1,aa,"[aa şey aa yani, ah aa bilirsin şey, aa yani t..."
2,aaa,"[aaa yoksa bu öyle, benimle aaa evlenir misin,..."
3,aaaah,[aaaah sadece bir saniyeliğine]
4,aaah,"[hiç aaah diye bağırıp, aaah diye bağırıp havaya]"
...,...,...
34299,şüphesiz,"[ben adamı çok güzel, hiç şüphesiz bu bir, şüp..."
34300,şüpheye,"[en ufak bir şüpheye, parkın güvenliği hakkınd..."
34301,şüpheyle,[şüpheyle yaklaşıyorum tabii kendi]
34302,şık,"[ne kadar da şık, bu çok şık bir, burası çok ş..."


In [48]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_five

,word,fivegram
0,bebeğimiz,birlikte bebeğimiz olsun istiyor musun
1,bebeğimiz,sadece sen ben ve bebeğimiz
2,ettiğim,rahatsız ettiğim için özür dilerim
3,ettiğim,rahatsız ettiğim için kusura bakmayın
4,ettiğim,rahatsız ettiğim için üzgünüm ama
...,...,...
606403,bastığı,bastığı zaman rahat etmez misiniz
606404,batıyoruz,batıyoruz ve gücümüz de yok
606405,ayrıntısına,şeyi en ince ayrıntısına kadar
606406,ayrıntısına,her şeyi en ince ayrıntısına


In [49]:
df_word_in_five_list = df_word_in_five.groupby(["word"])["fivegram"].apply(list).reset_index(name="fivegram")
df_word_in_five_list

,word,fivegram
0,a,"[take a woman like you, i t a r i, t a r i s, ..."
1,aa,"[şey aa yani tüm kasabaya, şey aa onlar yani ş..."
2,aaa,"[aaa yoksa bu öyle bir, aaa şey çünkü daha faz..."
3,aaah,"[hiç aaah diye bağırıp havaya, aaah diye bağır..."
4,aah,"[aah en uzun yolculuğuna daha, aah o gün olanl..."
...,...,...
27035,şüphesi,"[bundan hiç şüphesi oldu mu, hiç şüphesi oldu ..."
27036,şüphesiz,"[var ama onun için endişeleniyorum, şüphesiz b..."
27037,şüpheye,"[parkın güvenliği hakkında şüpheye düştü, oldu..."
27038,şık,"[şık bir isim değil mi, burası çok şık bir res..."


In [50]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_six

,word,sixgram
0,bildirin,çağrısı gönderip gemideki herkesin öldürüldüğü...
1,bildirin,kanalından durumunuzu bildirin ve kendinizi ta...
2,bildirin,kendinizi tanıtın ve pozisyonunuzu bildirin tamam
3,bildirin,tüm istasyona davetsiz misafirlerin geldiğini ...
4,bildirin,şüphelinin şu anki yerini bana bildirin
...,...,...
1781015,aştığını,bu tarz küçük duyguları aştığını düşünüyordum
1781016,aştığını,aştığını bilerek yüzümde bir gülümsemeyi zorlu...
1781017,incitmiş,onu incitmiş o da öç almak
1781018,incitmiş,birisi onu incitmiş o da öç


In [51]:
df_word_in_six_list = df_word_in_six.groupby(["word"])["sixgram"].apply(list).reset_index(name="sixgram")
df_word_in_six_list

,word,sixgram
0,a,"[a r i s y e, i t a r i s, t a r i s y, g i t ..."
1,aa,"[yani aa ahbap ya da adamım, oh bu sorun değil..."
2,aaa,"[aaa yoksa bu öyle bir resim, aaa şey buna ger..."
3,aaaah,[aaaah sadece bir saniyeliğine gözlerini açtı]
4,aaah,"[hiç aaah diye bağırıp havaya ateş, hayır hiç ..."
...,...,...
39759,şüpheyle,"[nasıl şüpheyle baktığını biliyorsun değil mi,..."
39760,şık,"[ama ikinci bölüm gerçekten çok şık, q bölümü ..."
39761,şıllık,[geri dön seni şişko sakallı şıllık]
39762,şımarık,"[aynen öyle kır hepsini şımarık gine, öyle kır..."


In [52]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_seven

,word,sevengram
0,geçtik,gelebilmek için pek çok duvarı yıkıp geçtik
1,geçtik,öteye geçtik mi şu tepeye doğru saldırmalıyız
2,geçtik,ondan öteye geçtik mi şu tepeye doğru
3,geçtik,biz bu belalı yoldan daha önce geçtik
4,geçtik,londradan trenle ayrıldık ve fırtınada kanalı ...
...,...,...
977329,yollayalım,için hava gücünün geri kalanını da yollayalım
977330,yollayalım,vücuduna bir kurşun sıkıp size cesedini yollay...
977331,aştığını,oldukça aştığını bilerek yüzümde bir gülümseme...
977332,incitmiş,onu incitmiş o da öç almak istiyor


In [53]:
df_word_in_seven_list = df_word_in_seven.groupby(["word"])["sevengram"].apply(list).reset_index(name="sevengram")
df_word_in_seven_list

,word,sevengram
0,a,"[i t a r i s y, t a r i s y e, a r i s y e n, ..."
1,aa,"[ve o aa şey aa yani tüm, aa bu resim bayan re..."
2,aaa,"[aaa yoksa bu öyle bir resim mi, aaa şey çünkü..."
3,aaah,"[aaah diye bağırıp havaya ateş ettin mi, hiç a..."
4,aah,"[aah en uzun yolculuğuna daha yeni başladın, a..."
...,...,...
35615,şüphesiz,"[şüphesiz ki bryan o bir baş belası, hiç şüphe..."
35616,şüpheye,[olduğum yerden gördüğüm kadarıyla şüpheye hiç...
35617,şüpheyle,[bize nasıl şüpheyle baktığını biliyorsun deği...
35618,şık,"[iş bulur bulmaz seni şehrin en şık, bulmaz se..."


In [54]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_eight

,word,eightgram
0,bunlara,kalbimi bunlara zorlamak için bir sebep var mı
1,bunlara,bunlara bakarken fark ettim ki kendimi iyi his...
2,bunlara,tüm bunlara bakarken fark ettim ki kendimi iyi
3,bunlara,hadi bütün bunlara son verebilecek biriyle kon...
4,bunlara,bütün bunlara son verebilecek biriyle konuşalı...
...,...,...
860070,yollayalım,iyisi vücuduna bir kurşun sıkıp size cesedini ...
860071,ayrıntısına,en ince ayrıntısına kadar bilen biri polisten ...
860072,ayrıntısına,ince ayrıntısına kadar bilen biri polisten bir...
860073,incitmiş,birisi onu incitmiş o da öç almak istiyor


In [55]:
df_word_in_eight_list = df_word_in_eight.groupby(["word"])["eightgram"].apply(list).reset_index(name="eightgram")
df_word_in_eight_list

,word,eightgram
0,a,"[i t a r i s y e, t a r i s y e n, g i t a r i..."
1,aa,"[ve o aa şey aa yani tüm kasabaya, kurbağa gib..."
2,aaa,"[aaa o da sütten çıkmış ak kaşık öyle, aaa eğe..."
3,aaah,[hayır hiç aaah diye bağırıp havaya ateş etmed...
4,aah,[lana aah bundan biraz daha karmaşık biliyor m...
...,...,...
35369,şüphesiz,[ama aramızda kalsın öldükten sonra artık hiçb...
35370,şüpheye,[olduğum yerden gördüğüm kadarıyla şüpheye hiç...
35371,şık,[iş bulur bulmaz seni şehrin en şık restoranın...
35372,şımarık,[seninle evlenirken bencil ve şımarık biri old...


In [56]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_nine

,word,ninegram
0,öpmek,beni öpmek zorunda değilsin ama ben seni öpmek...
1,istemedi,ya da onu içine soktuğu bu durumda yaşamak ist...
2,öpmek,yani ahbap onu öpmek istiyorsan hiç durma ve öp
3,istemedi,bir şey söylemek istemedi ama senin kazan oldu...
4,öpmek,ahbap onu öpmek istiyorsan hiç durma ve öp onu
...,...,...
355905,uygulanabilir,yaşına ve hastalığının ilerleme hızına bakarsa...
355906,uygulanabilir,ve hastalığının ilerleme hızına bakarsak bu du...
355907,yollayalım,ülkemizin çıkarları için hava gücünün geri kal...
355908,yollayalım,en iyisi vücuduna bir kurşun sıkıp size cesedi...


In [57]:
df_word_in_nine_list = df_word_in_nine.groupby(["word"])["ninegram"].apply(list).reset_index(name="ninegram")
df_word_in_nine_list

,word,ninegram
0,a,"[i t a r i s y e n, g i t a r i s y e, e g o m..."
1,aa,[şey bu kadar basit olmayabilir belkide yani a...
2,aaa,"[aaa o da sütten çıkmış ak kaşık öyle mi, aaa ..."
3,aah,[oh uh evet uh ben buradayım saygıdeğer efendi...
4,abartılı,[ve sanıyorum onların seni sevdiği kısmı biraz...
...,...,...
27724,şüphesiz,[şüphesiz ki yapamazsan daha başlamadan yenilg...
27725,şüpheye,[en ufak bir şüpheye yer olmadan birini böyles...
27726,şık,[iş bulur bulmaz seni şehrin en şık restoranın...
27727,şımarık,[siz güvensiz şımarık ve erkeklerin çekici bul...


In [58]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_ten

,word,tengram
0,etsen,acele etsen iyi olur çünkü o bizi teker teker ...
1,etsen,ismini falan vermeden rapor etsen yeterli olur...
2,etsen,kabul etsen de etmesen de buraya bir şey için ...
3,etsen,laflarına dikkat etsen iyi edersin yoksa başka...
4,etsen,şimdi terk etsen bile beni sahip olduğum tek ş...
...,...,...
298250,wexler,ozzie wexler adlı bir bilim adamı bugün alışve...
298251,uygulanabilir,yaşına ve hastalığının ilerleme hızına bakarsa...
298252,uygulanabilir,gurur ve utanç çalışanlara karşı değil üzerind...
298253,kardeşisin,kralın kardeşisin demek ona rağmen ordusuna se...


In [59]:
df_word_in_ten_list = df_word_in_ten.groupby(["word"])["tengram"].apply(list).reset_index(name="tengram")
df_word_in_ten_list

,word,tengram
0,a,"[g i t a r i s y e n, a takımı değil belki ama..."
1,aa,[yani şey bu kadar basit olmayabilir belkide y...
2,aah,[bir fbi programına virüs koyup programı iki y...
3,ab,[genç kız tedavi edici ab için gayet sağlıklı ...
4,abartmış,[halk tipi imajını abartmış ama istese tırnakl...
...,...,...
28380,şüphesi,[bundan kimsenin şüphesi yok ama burada çiftli...
28381,şüphesiz,[şüphesiz erdemli bir lider sadece küçük bir a...
28382,şüpheye,[onun beni şüpheye yer vermeyecek şekilde gözl...
28383,şık,[bir iş bulur bulmaz seni şehrin en şık restor...


In [60]:
dfs = [df_word_select, df_word_in_two_list, df_word_in_three_list, df_word_in_four_list, df_word_in_five_list, df_word_in_six_list, df_word_in_seven_list, df_word_in_eight_list, df_word_in_nine_list, df_word_in_ten_list]

In [61]:
df_word_merge = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='left'), dfs)
df_word_merge

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir,18835735,"['bir şey', 'bu bir', 'bir dakika', 'bir şeyle...","[bir şey yok, bir şey var, başka bir şey, bir ...","[başka bir bebeğimiz olacak, anda gondor topra...","[başka bir şey var mı, sana bir şey sorabilir ...","[sana söylemem gereken bir şey var, bir iki üç...","[bir iki üç dört beş altı yedi, bana söylemek ...","[bir iki üç dört beş altı yedi sekiz, sekiz ye...","[dokuz sekiz yedi altı beş dört üç iki bir, bi...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,bu,11062659,"['bu kadar', 'bu da', 'bu çok', 'bu bir', 'bu ...","[bu da ne, neden bu kadar, bu doğru değil, bu ...","[bu da ne demek, bu da ne böyle, bu iyi bir fi...","[bu da ne demek oluyor, bu da ne demek şimdi, ...","[bu iyi bir şey değil mi, bence bu iyi bir fik...","[bu sana bir şey ifade ediyor mu, evime varmay...","[bu sabah uyandın bütün o sevgi yok olmuş, bu ...",[bu gün bir kez daha biri bir yerlerde ağlayac...,[bu işlemi bu akşam başlatmak istiyorsak yapac...
2,ne,8025880,"['ne oldu', 'ne kadar', 'ne yapıyorsun', 'ne d...","[bu da ne, ne demek istiyorsun, ne işin var, n...","[burada ne işin var, bu da ne demek, en son ne...","[bu da ne demek oluyor, bu da ne demek şimdi, ...","[onu en son ne zaman gördün, bunun ne anlama g...",[senin için ne kadar önemli olduğunu biliyorum...,[bunun senin için ne kadar zor olduğunu biliyo...,[efendi luke sizi tekrar işler vaziyette görme...,[ne dediğimi duymadın mı evlat ben de ona evet...
3,ve,7766036,"['ve ben', 've bu', 've bir', 'sen ve', 've se...","[sen ve ben, bayanlar ve baylar, ve ben de, ve...","[sadece sen ve ben, oğul ve kutsal ruh, baba o...","[baba oğul ve kutsal ruh, oğul ve kutsal ruh a...","[baba oğul ve kutsal ruh adına, evrenin her ya...","[bir ve iki ve üç ve dört, ve sen herkesin sen...","[ve sen herkesin seni tanıdığı o yerde olmak, ...",[ve sen herkesin seni tanıdığı o yerde olmak i...,[zamanımızı geri almak istiyorum ama şans ve k...
4,için,5484109,"['senin için', 'benim için', 'için bir', 'bunu...","[için özür dilerim, için teşekkür ederim, beni...","[sizin için ne yapabilirim, senin için ne yapa...","[rahatsız ettiğim için özür dilerim, senin içi...","[için yapabileceğim bir şey var mı, benim için...","[senin için yapabileceğim bir şey var mı, sizi...",[sizin için yapabileceğim başka bir şey var mı...,[buralara kadar gelebilmek için pek çok duvarı...,[annem bana bazı şeyleri anlatmak için mutlaka...
...,...,...,...,...,...,...,...,...,...,...,...
44995,hayırbu,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44996,konferansına,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44997,kardeşisin,871,"['onun kardeşisin', 'kız kardeşisin']","[kardeşisin değil mi, onun kız kardeşisin]","[de onun kız kardeşisin, kız kardeşisin değil mi]",[sen de onun kız kardeşisin],NaN,NaN,NaN,NaN,[kralın kardeşisin demek ona rağmen ordusuna s...
44998,kaldırıyorsun,871,['kadar kaldırıyorsun'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_word_merge.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_word_text = pd.read_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx")

In [ ]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

In [ ]:
df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

##### Twogram

In [23]:
# 1 Way Slow
#df_two_in_three = word_in_wordgroup(twogram_list, df_threegram_select, "threegram", simple=True)
#df_two_in_three.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_three

In [24]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_three = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_three.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_three

,twogram,threegram
0,korkutmak istemedim,seni korkutmak istemedim
1,korkutmak istemedim,sizi korkutmak istemedim
2,farkında değilsin,yaptığının farkında değilsin
3,farkında değilsin,olduğunun farkında değilsin
4,farkında değilsin,dediğinin farkında değilsin
...,...,...
557791,araba şirketi,araba şirketi için
557792,bir ederim,yerle bir ederim
557793,kaşık daha,bir kaşık daha
557794,olmadan gerçek,olmadan gerçek bir


In [25]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

,twogram,threegram
0,a artı,[a artı b]
1,a b,"[a b c, a b ve]"
2,a de,[l a de]
3,a e,[a e i]
4,a ile,"[a ile başlayan, a ile başlıyor]"
...,...,...
128721,şık bir,"[çok şık bir, şık bir yer, şık bir şey, şık bi..."
128722,şık değil,"[çok şık değil, şık değil mi]"
128723,şık görünüyorsun,[çok şık görünüyorsun]
128724,şık ve,[çok şık ve]


In [26]:
#df_two_in_three_list.to_excel("Two_In_Three_List.xlsx", index=False)

In [ ]:
#df_two_in_four = word_in_wordgroup(twogram_list, df_fourgram_select, "fourgram", simple=True)
#df_two_in_four.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_four

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_four

In [ ]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

In [ ]:
#df_two_in_five = word_in_wordgroup(twogram_list, df_fivegram_select, "fivegram", simple=True)
#df_two_in_five.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_five

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
#df_two_in_six = word_in_wordgroup(twogram_list, df_sixgram_select, "sixgram", simple=True)
#df_two_in_six.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_six

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
#df_two_in_seven = word_in_wordgroup(twogram_list, df_sevengram_select, "sevengram", simple=True)
#df_two_in_seven.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_seven

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
#df_two_in_eight = word_in_wordgroup(twogram_list, df_eightgram_select, "eightgram", simple=True)
#df_two_in_eight.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_eight

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
#df_two_in_nine = word_in_wordgroup(twogram_list, df_ninegram_select, "ninegram", simple=True)
#df_two_in_nine.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_nine

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
#df_two_in_ten = word_in_wordgroup(twogram_list, df_tengram_select, "tengram", simple=True)
#df_two_in_ten.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_ten

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)

##### Threegram

In [ ]:
#df_three_in_four = word_in_wordgroup(threegram_list, df_fourgram_select, "fourgram", simple=True)
#df_three_in_four.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_four

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
#df_three_in_five = word_in_wordgroup(threegram_list, df_fivegram_select, "fivegram", simple=True)
#df_three_in_five.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_five

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
#df_three_in_six = word_in_wordgroup(threegram_list, df_sixgram_select, "sixgram", simple=True)
#df_three_in_six.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_six

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
#df_three_in_seven = word_in_wordgroup(threegram_list, df_sevengram_select, "sevengram", simple=True)
#df_three_in_seven.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_seven

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
#df_three_in_eight = word_in_wordgroup(threegram_list, df_eightgram_select, "eightgram", simple=True)
#df_three_in_eight.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_eight

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
#df_three_in_nine = word_in_wordgroup(threegram_list, df_ninegram_select, "ninegram", simple=True)
#df_three_in_nine.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_nine

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
#df_three_in_ten = word_in_wordgroup(threegram_list, df_tengram_select, "tengram", simple=True)
#df_three_in_ten.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_ten

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)

##### Fourgram

In [ ]:
#df_four_in_five = word_in_wordgroup(fourgram_list, df_fivegram_select, "fivegram", simple=True)
#df_four_in_five.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_five

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
#df_four_in_six = word_in_wordgroup(fourgram_list, df_sixgram_select, "sixgram", simple=True)
#df_four_in_six.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_six

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
#df_four_in_seven = word_in_wordgroup(fourgram_list, df_sevengram_select, "sevengram", simple=True)
#df_four_in_seven.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_seven

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
#df_four_in_eight = word_in_wordgroup(fourgram_list, df_eightgram_select, "eightgram", simple=True)
#df_four_in_eight.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_eight

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
#df_four_in_nine = word_in_wordgroup(fourgram_list, df_ninegram_select, "ninegram", simple=True)
#df_four_in_nine.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_nine

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
#df_four_in_ten = word_in_wordgroup(fourgram_list, df_tengram_select, "tengram", simple=True)
#df_four_in_ten.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_ten

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)

##### Fivegram

In [ ]:
#df_five_in_six = word_in_wordgroup(fivegram_list, df_sixgram_select, "sixgram", simple=True)
#df_five_in_six.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_six

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
#df_five_in_seven = word_in_wordgroup(fivegram_list, df_sevengram_select, "sevengram", simple=True)
#df_five_in_seven.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_seven

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
#df_five_in_eight = word_in_wordgroup(fivegram_list, df_eightgram_select, "eightgram", simple=True)
#df_five_in_eight.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_eight

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
#df_five_in_nine = word_in_wordgroup(fivegram_list, df_ninegram_select, "ninegram", simple=True)
#df_five_in_nine.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_nine

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
#df_five_in_ten = word_in_wordgroup(fivegram_list, df_tengram_select, "tengram", simple=True)
#df_five_in_ten.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_ten

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)

##### Sixgram

In [ ]:
#df_six_in_seven = word_in_wordgroup(sixgram_list, df_sevengram_select, "sevengram", simple=True)
#df_six_in_seven.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_seven

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
#df_six_in_eight = word_in_wordgroup(sixgram_list, df_eightgram_select, "eightgram", simple=True)
#df_six_in_eight.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_eight

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
#df_six_in_nine = word_in_wordgroup(sixgram_list, df_ninegram_select, "ninegram", simple=True)
#df_six_in_nine.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_nine

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
#df_six_in_ten = word_in_wordgroup(sixgram_list, df_tengram_select, "tengram", simple=True)
#df_six_in_ten.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_ten

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)

##### Sevengram

In [ ]:
#df_seven_in_eight = word_in_wordgroup(sevengram_list, df_eightgram_select, "eightgram", simple=True)
#df_seven_in_eight.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_eight

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_eight

In [ ]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

In [ ]:
#df_seven_in_nine = word_in_wordgroup(sevengram_list, df_ninegram_select, "ninegram", simple=True)
#df_seven_in_nine.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_nine

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_nine

In [ ]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

In [ ]:
#df_seven_in_ten = word_in_wordgroup(sevengram_list, df_tengram_select, "tengram", simple=True)
#df_seven_in_ten.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_ten

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_ten

In [ ]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

In [ ]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [ ]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

In [ ]:
df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)

##### Eightgram

In [ ]:
#df_eight_in_nine = word_in_wordgroup(eightgram_list, df_ninegram_select, "ninegram", simple=True)
#df_eight_in_nine.rename(columns={"word":"eightgram"}, inplace=True)
#df_eight_in_nine

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_eight_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_eight_in_nine

In [ ]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

In [ ]:
#df_eight_in_ten = word_in_wordgroup(eightgram_list, df_tengram_select, "tengram", simple=True)
#df_eight_in_ten.rename(columns={"word":"eightgram"}, inplace=True)
#df_eight_in_ten

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_eight_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_eight_in_ten

In [ ]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

In [ ]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [ ]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

In [ ]:
df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)

##### Ninegram

In [ ]:
#df_nine_in_ten = word_in_wordgroup(ninegram_list, df_tengram_select, "tengram", simple=True)
#df_nine_in_ten.rename(columns={"word":"ninegram"}, inplace=True)
#df_nine_in_ten

In [ ]:
# parameter
source_wordgroup_list = ninegram_list
search_column_name = "ninegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_nine_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_nine_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_nine_in_ten

In [ ]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

In [ ]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [ ]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

In [ ]:
df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)

#### Copy Move And Delete

In [111]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*.xlsx")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Word_Text_Selection_Result3.xlsx']

In [112]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [113]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [ ]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
word_list = df_word_select["word"].to_list()

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])

In [92]:
df_word_text = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Text Selection/Data/2-Text Selection Data Merge/Turkish_Word_Text_Selection_Result2.xlsx")
df_word_text

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir,18835735,"['bir şey', 'bu bir', 'bir dakika', 'bir şeyle...","['bir şey yok', 'bir şey var', 'başka bir şey'...","['başka bir bebeğimiz olacak', 'anda gondor to...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'bir iki...","['bir iki üç dört beş altı yedi', 'bana söylem...","['bir iki üç dört beş altı yedi sekiz', 'sekiz...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,bu,11062659,"['bu kadar', 'bu da', 'bu çok', 'bu bir', 'bu ...","['bu da ne', 'neden bu kadar', 'bu doğru değil...","['bu da ne demek', 'bu da ne böyle', 'bu iyi b...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu iyi bir şey değil mi', 'bence bu iyi bir ...","['bu sana bir şey ifade ediyor mu', 'evime var...","['bu sabah uyandın bütün o sevgi yok olmuş', '...",['bu gün bir kez daha biri bir yerlerde ağlaya...,['bu işlemi bu akşam başlatmak istiyorsak yapa...
2,ne,8025880,"['ne oldu', 'ne kadar', 'ne yapıyorsun', 'ne d...","['bu da ne', 'ne demek istiyorsun', 'ne işin v...","['burada ne işin var', 'bu da ne demek', 'en s...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['onu en son ne zaman gördün', 'bunun ne anlam...",['senin için ne kadar önemli olduğunu biliyoru...,['bunun senin için ne kadar zor olduğunu biliy...,['efendi luke sizi tekrar işler vaziyette görm...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,ve,7766036,"['ve ben', 've bu', 've bir', 'sen ve', 've se...","['sen ve ben', 'bayanlar ve baylar', 've ben d...","['sadece sen ve ben', 'oğul ve kutsal ruh', 'b...","['baba oğul ve kutsal ruh', 'oğul ve kutsal ru...","['baba oğul ve kutsal ruh adına', 'evrenin her...","['bir ve iki ve üç ve dört', 've sen herkesin ...",['ve sen herkesin seni tanıdığı o yerde olmak'...,['ve sen herkesin seni tanıdığı o yerde olmak ...,['zamanımızı geri almak istiyorum ama şans ve ...
4,için,5484109,"['senin için', 'benim için', 'için bir', 'bunu...","['için özür dilerim', 'için teşekkür ederim', ...","['sizin için ne yapabilirim', 'senin için ne y...","['rahatsız ettiğim için özür dilerim', 'senin ...","['için yapabileceğim bir şey var mı', 'benim i...","['senin için yapabileceğim bir şey var mı', 's...",['sizin için yapabileceğim başka bir şey var m...,['buralara kadar gelebilmek için pek çok duvar...,['annem bana bazı şeyleri anlatmak için mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
44995,hayırbu,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44996,konferansına,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44997,kardeşisin,871,"['onun kardeşisin', 'kız kardeşisin']","['kardeşisin değil mi', 'onun kız kardeşisin']","['de onun kız kardeşisin', 'kız kardeşisin değ...",['sen de onun kız kardeşisin'],NaN,NaN,NaN,NaN,['kralın kardeşisin demek ona rağmen ordusuna ...
44998,kaldırıyorsun,871,['kadar kaldırıyorsun'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir,18835735,"['bir şey', 'bu bir', 'bir dakika', 'bir şeyle...","['bir şey yok', 'bir şey var', 'başka bir şey'...","['başka bir bebeğimiz olacak', 'anda gondor to...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'bir iki...","['bir iki üç dört beş altı yedi', 'bana söylem...","['bir iki üç dört beş altı yedi sekiz', 'sekiz...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,bu,11062659,"['bu kadar', 'bu da', 'bu çok', 'bu bir', 'bu ...","['bu da ne', 'neden bu kadar', 'bu doğru değil...","['bu da ne demek', 'bu da ne böyle', 'bu iyi b...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu iyi bir şey değil mi', 'bence bu iyi bir ...","['bu sana bir şey ifade ediyor mu', 'evime var...","['bu sabah uyandın bütün o sevgi yok olmuş', '...",['bu gün bir kez daha biri bir yerlerde ağlaya...,['bu işlemi bu akşam başlatmak istiyorsak yapa...
2,ne,8025880,"['ne oldu', 'ne kadar', 'ne yapıyorsun', 'ne d...","['bu da ne', 'ne demek istiyorsun', 'ne işin v...","['burada ne işin var', 'bu da ne demek', 'en s...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['onu en son ne zaman gördün', 'bunun ne anlam...",['senin için ne kadar önemli olduğunu biliyoru...,['bunun senin için ne kadar zor olduğunu biliy...,['efendi luke sizi tekrar işler vaziyette görm...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,ve,7766036,"['ve ben', 've bu', 've bir', 'sen ve', 've se...","['sen ve ben', 'bayanlar ve baylar', 've ben d...","['sadece sen ve ben', 'oğul ve kutsal ruh', 'b...","['baba oğul ve kutsal ruh', 'oğul ve kutsal ru...","['baba oğul ve kutsal ruh adına', 'evrenin her...","['bir ve iki ve üç ve dört', 've sen herkesin ...",['ve sen herkesin seni tanıdığı o yerde olmak'...,['ve sen herkesin seni tanıdığı o yerde olmak ...,['zamanımızı geri almak istiyorum ama şans ve ...
4,için,5484109,"['senin için', 'benim için', 'için bir', 'bunu...","['için özür dilerim', 'için teşekkür ederim', ...","['sizin için ne yapabilirim', 'senin için ne y...","['rahatsız ettiğim için özür dilerim', 'senin ...","['için yapabileceğim bir şey var mı', 'benim i...","['senin için yapabileceğim bir şey var mı', 's...",['sizin için yapabileceğim başka bir şey var m...,['buralara kadar gelebilmek için pek çok duvar...,['annem bana bazı şeyleri anlatmak için mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
44993,aştığını,871,NaN,NaN,NaN,NaN,['oldukça aştığını bilerek yüzümde bir gülümse...,['oldukça aştığını bilerek yüzümde bir gülümse...,NaN,NaN,NaN
44994,incitmiş,871,NaN,NaN,NaN,NaN,"['onu incitmiş o da öç almak', 'birisi onu inc...","['onu incitmiş o da öç almak istiyor', 'birisi...","['birisi onu incitmiş o da öç almak istiyor', ...",NaN,NaN
44997,kardeşisin,871,"['onun kardeşisin', 'kız kardeşisin']","['kardeşisin değil mi', 'onun kız kardeşisin']","['de onun kız kardeşisin', 'kız kardeşisin değ...",['sen de onun kız kardeşisin'],NaN,NaN,NaN,NaN,['kralın kardeşisin demek ona rağmen ordusuna ...
44998,kaldırıyorsun,871,['kadar kaldırıyorsun'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result3.xlsx", index=False)